In [25]:
# import package
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt

# set numpy seed for reproducibility
np.random.seed(seed=123)

In [44]:
dataset = [[1.0, 1, -1, -1],
          [-1, -1, 1, 1],
          [-1, -1, 1, 1],
          [1, 1, -1, -1],
          [1, 0, -1, -1],
          [1, 1, 0, -1],
          [0, 1, -1, -1],
          [1, 1, -1, 0]]

In [45]:
x_train = dataset
y_train = dataset

In [46]:
batch_size = len(x_train) # 50
num_epochs = 100
learning_rate = 0.01
size_hidden = 100
n_feature = 4
n_output = 4
batch_no = int(len(x_train) / batch_size)

In [127]:
from torch import nn
from torch.autograd import Variable

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(4, 100),
            nn.ReLU(),
            nn.Linear(100, 25))
        self.decoder = nn.Sequential(
            nn.Linear(25, 100),
            nn.ReLU(),
            nn.Linear(100, 4))

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

net = autoencoder()

In [128]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-5)

In [129]:
def denoising(input_tensor, output_tensor):
    for i in range(len(input_tensor)):
        for j in range(len(input_tensor[i])):
            if float(input_tensor[i][j]) == 0.0:
                output_tensor[i][j] = 0.0
    return output_tensor

In [130]:
x = torch.tensor([[1, 1, 0, 1, 0, 1], [1, 0, 0, 1, 1, 1]])
y = torch.tensor([[1.2, 0.8, 0.9, 2.3, 3.2, 1.8], [1, 0.8, 0.9, 2, 3, 1]])
denoising(x, y)

tensor([[1.2000, 0.8000, 0.0000, 2.3000, 0.0000, 1.8000],
        [1.0000, 0.0000, 0.0000, 2.0000, 3.0000, 1.0000]])

In [131]:
running_loss = 0
for epoch in range(num_epochs):
    
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.tensor(x_train[start:end]).float())
        labels = Variable(torch.tensor(y_train[start:end]).float())
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        
        # Start DAE
        dae_outputs = denoising(inputs, outputs)
        # End DAE
        
        #print("outputs",outputs)
        #print("outputs",outputs,outputs.shape,"labels",labels, labels.shape)
        loss = criterion(dae_outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    if epoch % 10 == 0:    
        print('Epoch {}'.format(epoch+1), "loss: ", running_loss)
    running_loss = 0.0

Epoch 1 loss:  0.9272454977035522
Epoch 11 loss:  0.013974918983876705
Epoch 21 loss:  0.006095705553889275
Epoch 31 loss:  0.0014510368928313255
Epoch 41 loss:  0.0006224928074516356
Epoch 51 loss:  0.0002630316885188222
Epoch 61 loss:  0.0001008296458167024
Epoch 71 loss:  7.84692310844548e-05
Epoch 81 loss:  3.5212753573432565e-05
Epoch 91 loss:  1.63125223480165e-05


In [132]:
print (inputs)
print (outputs)

tensor([[ 1.,  1., -1., -1.],
        [-1., -1.,  1.,  1.],
        [-1., -1.,  1.,  1.],
        [ 1.,  1., -1., -1.],
        [ 1.,  0., -1., -1.],
        [ 1.,  1.,  0., -1.],
        [ 0.,  1., -1., -1.],
        [ 1.,  1., -1.,  0.]])
tensor([[ 0.9968,  1.0032, -0.9965, -1.0052],
        [-1.0001, -0.9979,  1.0011,  0.9995],
        [-1.0001, -0.9979,  1.0011,  0.9995],
        [ 0.9968,  1.0032, -0.9965, -1.0052],
        [ 1.0050,  0.0000, -1.0006, -0.9952],
        [ 1.0038,  0.9948,  0.0000, -0.9973],
        [ 0.0000,  0.9921, -1.0055, -0.9978],
        [ 1.0001,  0.9968, -1.0004,  0.0000]], grad_fn=<CopySlices>)


In [133]:
print (inputs)
print (outputs)

tensor([[ 1.,  1., -1., -1.],
        [-1., -1.,  1.,  1.],
        [-1., -1.,  1.,  1.],
        [ 1.,  1., -1., -1.],
        [ 1.,  0., -1., -1.],
        [ 1.,  1.,  0., -1.],
        [ 0.,  1., -1., -1.],
        [ 1.,  1., -1.,  0.]])
tensor([[ 0.9968,  1.0032, -0.9965, -1.0052],
        [-1.0001, -0.9979,  1.0011,  0.9995],
        [-1.0001, -0.9979,  1.0011,  0.9995],
        [ 0.9968,  1.0032, -0.9965, -1.0052],
        [ 1.0050,  0.0000, -1.0006, -0.9952],
        [ 1.0038,  0.9948,  0.0000, -0.9973],
        [ 0.0000,  0.9921, -1.0055, -0.9978],
        [ 1.0001,  0.9968, -1.0004,  0.0000]], grad_fn=<CopySlices>)


In [134]:
pred = net(Variable(torch.tensor([[1, 0, 0, 0]]).float()))
print (pred.data)

tensor([[ 0.6571,  0.6047, -0.6330, -0.6289]])


In [135]:
pred = net(Variable(torch.tensor([[1, 0, 0, 0]]).float()))
print (pred.data)

tensor([[ 0.6571,  0.6047, -0.6330, -0.6289]])


In [136]:
pred = net(Variable(torch.tensor([[0, 1, 0, 0]]).float()))
print (pred.data)

tensor([[ 0.7072,  0.7084, -0.6899, -0.7191]])


In [137]:
pred = net(Variable(torch.tensor([[0, 0, -1, 0]]).float()))
print (pred.data)

tensor([[ 0.6548,  0.6414, -0.6987, -0.6681]])


In [138]:
pred = net(Variable(torch.tensor([[0, 0, 0, -1]]).float()))
print (pred.data)

tensor([[ 0.6546,  0.6299, -0.6408, -0.6676]])


In [126]:
outputs[0][1]

tensor(1.0035, grad_fn=<SelectBackward>)

In [100]:
outputs[0][1]

tensor(0.9962, grad_fn=<SelectBackward>)